# CIFAR-10

In [1]:
from matrix_square_root_power import *
from shampoo_optimizer import *
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, Sequential
from keras.datasets import cifar10
from keras import backend
from keras.utils import np_utils
import numpy
from keras.datasets import cifar10
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [0]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
test_images = test_images / 255.0

In [0]:
def loss(model, x, y, loss_object):
    y_ = model(x)

    return loss_object(y_true=y, y_pred=y_)

def grad(model, inputs, targets, loss_object):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, loss_object)
        return loss_value, tape.gradient(loss_value, model.trainable_weights)

In [0]:
train_images = tf.convert_to_tensor(train_images, dtype=tf.float32)
test_images = tf.convert_to_tensor(test_images, dtype=tf.float32)

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

shampoo_train_loss_results = []
shampoo_train_accuracy_results = []

epochs = 50
batch_size = 128

for epoch in range(epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    # Training loop - using batches of 128
    for i in range(train_images.shape[0] // batch_size):
        x = train_images[i*batch_size:i*batch_size+batch_size]
        y = train_labels[i*batch_size:i*batch_size+batch_size]
        # Optimize the model
        loss_value, grads = grad(model, x, y, loss_object)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Track progress
        epoch_loss_avg(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        epoch_accuracy(y, model(x))

        # End epoch
    shampoo_train_loss_results.append(epoch_loss_avg.result())
    shampoo_train_accuracy_results.append(epoch_accuracy.result())

    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 2.134, Accuracy: 32.957%
Epoch 001: Loss: 2.035, Accuracy: 43.606%
Epoch 002: Loss: 1.983, Accuracy: 48.870%
Epoch 003: Loss: 1.945, Accuracy: 53.017%
Epoch 004: Loss: 1.921, Accuracy: 55.663%
Epoch 005: Loss: 1.893, Accuracy: 58.468%
Epoch 006: Loss: 1.878, Accuracy: 60.006%
Epoch 007: Loss: 1.864, Accuracy: 61.432%
Epoch 008: Loss: 1.851, Accuracy: 62.786%
Epoch 009: Loss: 1.837, Accuracy: 64.151%
Epoch 010: Loss: 1.827, Accuracy: 65.182%
Epoch 011: Loss: 1.815, Accuracy: 66.464%
Epoch 012: Loss: 1.807, Accuracy: 67.175%
Epoch 013: Loss: 1.796, Accuracy: 68.353%
Epoch 014: Loss: 1.791, Accuracy: 68.760%
Epoch 015: Loss: 1.782, Accuracy: 69.804%
Epoch 016: Loss: 1.777, Accuracy: 70.246%
Epoch 017: Loss: 1.770, Accuracy: 70.933%
Epoch 018: Loss: 1.764, Accuracy: 71.534%
Epoch 019: Loss: 1.753, Accuracy: 72.588%
Epoch 020: Loss: 1.752, Accuracy: 72.692%
Epoch 021: Loss: 1.745, Accuracy: 73.369%
Epoch 022: Loss: 1.739, Accuracy: 73.992%
Epoch 023: Loss: 1.734, Accuracy: 